In [1]:
import scipy.io as sio                     
import numpy as np                         
import matplotlib.pyplot as plt 
import os  
os.chdir('/home/yifeishen/GNN-Resource-Management/D2D/Weighted Power Control')        
import function_wmmse_powercontrol as wf
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Sigmoid, BatchNorm1d as BN
import wireless_networks_generator as wg
from FPLinQ import FP_optimize, FP
import helper_functions
import time

class init_parameters():
    def __init__(self):
        # wireless network settings
        self.n_links = train_K
        self.field_length = 1000
        self.shortest_directLink_length = 2
        self.longest_directLink_length = 65
        self.shortest_crossLink_length = 1
        self.bandwidth = 5e6
        self.carrier_f = 2.4e9
        self.tx_height = 1.5
        self.rx_height = 1.5
        self.antenna_gain_decibel = 2.5
        self.tx_power_milli_decibel = 40
        self.tx_power = np.power(10, (self.tx_power_milli_decibel-30)/10)
        self.noise_density_milli_decibel = -169
        self.input_noise_power = np.power(10, ((self.noise_density_milli_decibel-30)/10)) * self.bandwidth
        self.output_noise_power = self.input_noise_power
        self.SNR_gap_dB = 6
        self.SNR_gap = np.power(10, self.SNR_gap_dB/10)
        self.setting_str = "{}_links_{}X{}_{}_{}_length".format(self.n_links, self.field_length, self.field_length, self.shortest_directLink_length, self.longest_directLink_length)
        # 2D occupancy grid setting
        self.cell_length = 5
        self.n_grids = np.round(self.field_length/self.cell_length).astype(int)

def proc_train_losses(train_path_losses, train_channel_losses):
    mask = np.eye(train_K)
    diag_path = np.multiply(mask,train_path_losses)
    off_diag_path = train_path_losses - diag_path
    diag_channel = np.multiply(mask,train_channel_losses)
    train_losses = diag_channel + off_diag_path
    return train_losses

def normalize_data(train_data,test_data):
    mask = np.eye(train_K)
    train_copy = np.copy(train_data)
    diag_H = np.multiply(mask,train_copy)
    diag_mean = np.sum(diag_H)/train_layouts/train_K
    diag_var = np.sqrt(np.sum(np.square(diag_H))/train_layouts/train_K)
    tmp_diag = (diag_H - diag_mean)/diag_var

    off_diag = train_copy - diag_H
    off_diag_mean = np.sum(off_diag)/train_layouts/train_K/(train_K-1)
    off_diag_var = np.sqrt(np.sum(np.square(off_diag))/train_layouts/train_K/(train_K-1))
    tmp_off = (off_diag - off_diag_mean)/off_diag_var
    tmp_off_diag = tmp_off - np.multiply(tmp_off,mask)
    
    norm_train = np.multiply(tmp_diag,mask) + tmp_off_diag
    
    # normlize test
    mask = np.eye(test_K)
    test_copy = np.copy(test_data)
    diag_H = np.multiply(mask,test_copy)
    tmp_diag = (diag_H - diag_mean)/diag_var
    
    off_diag = test_copy - diag_H
    tmp_off = (off_diag - off_diag_mean)/off_diag_var
    tmp_off_diag = tmp_off - np.multiply(tmp_off,mask)
    
    norm_test = np.multiply(tmp_diag,mask) + tmp_off_diag
    
    return norm_train, norm_test

def simple_greedy(X,AAA,label):
    
    n = X.shape[0]
    thd = int(np.sum(label)/n)
    Y = np.zeros((n,test_K))
    for ii in range(n):
        alpha = AAA[ii,:]
        H_diag = alpha * np.square(np.diag(X[ii,:,:]))
        xx = np.argsort(H_diag)[::-1]
        for jj in range(thd):
            Y[ii,xx[jj]] = 1
    return Y

def build_graph(loss, dist, norm_dist, norm_loss, K, threshold):
    n = loss.shape[0]
    x1 = np.expand_dims(np.diag(norm_dist),axis=1)
    x2 = np.expand_dims(np.diag(norm_loss),axis=1)
    x3 = np.ones((K,1))
    x = np.concatenate((x1,x2,x3),axis=1)
    x = torch.tensor(x, dtype=torch.float)
    
    dist2 = np.copy(dist)
    mask = np.eye(K)
    diag_dist = np.multiply(mask,dist2)
    dist2 = dist2 + 1000* diag_dist
    dist2[dist2 > threshold] = 0
    attr_ind = np.nonzero(dist2)
    e1 = np.expand_dims(norm_loss[attr_ind],  axis = -1)
    e2 = np.expand_dims((norm_loss.transpose(0,1))[attr_ind],  axis = -1)
    e3 = np.expand_dims(norm_dist[attr_ind],  axis = -1)
    edge_attr = np.concatenate((e1,e2,e3),axis=-1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    
    attr_ind = np.array(attr_ind)
    adj = np.zeros(attr_ind.shape)
    adj[0,:] = attr_ind[1,:]
    adj[1,:] = attr_ind[0,:]
    edge_index = torch.tensor(adj, dtype=torch.long)

    y = torch.tensor(np.expand_dims(loss,axis=0), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.contiguous(),edge_attr = edge_attr, y = y)
    return data

def proc_data(HH, dists, norm_dists, norm_HH, K):
    n = HH.shape[0]
    data_list = []
    for i in range(n):
        data = build_graph(HH[i,:,:],dists[i,:,:], norm_dists[i,:,:], norm_HH[i,:,:], K,300)
        data_list.append(data)
    return data_list

class IGConv(MessagePassing):
    def __init__(self, mlp1, mlp2, **kwargs):
        super(IGConv, self).__init__(aggr='max', **kwargs)

        self.mlp1 = mlp1
        self.mlp2 = mlp2
        #self.reset_parameters()

    def reset_parameters(self):
        reset(self.mlp1)
        reset(self.mlp2)
        
    def update(self, aggr_out, x):
        tmp = torch.cat([x, aggr_out], dim=1)
        comb = self.mlp2(tmp)
        return torch.cat([x[:,:2], comb],dim=1)
        
    def forward(self, x, edge_index, edge_attr):
        x = x.unsqueeze(-1) if x.dim() == 1 else x
        edge_attr = edge_attr.unsqueeze(-1) if edge_attr.dim() == 1 else edge_attr
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr):
        tmp = torch.cat([x_j, edge_attr], dim=1)
        agg = self.mlp1(tmp)
        return agg

    def __repr__(self):
        return '{}(nn={})'.format(self.__class__.__name__, self.mlp1,self.mlp2)

def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU())#, BN(channels[i]))
        for i in range(1, len(channels))
    ])
class IGCNet(torch.nn.Module):
    def __init__(self):
        super(IGCNet, self).__init__()

        self.mlp1 = MLP([6, 32, 32])
        self.mlp2 = MLP([35, 16])
        self.mlp2 = Seq(*[self.mlp2,Seq(Lin(16, 1), Sigmoid())])
        self.conv = IGConv(self.mlp1,self.mlp2)

    def forward(self, data):
        x0, edge_attr, edge_index = data.x, data.edge_attr, data.edge_index
        x1 = self.conv(x = x0, edge_index = edge_index, edge_attr = edge_attr)
        x2 = self.conv(x = x1, edge_index = edge_index, edge_attr = edge_attr)
        #x3 = self.conv(x = x2, edge_index = edge_index, edge_attr = edge_attr)
        #x4 = self.conv(x = x3, edge_index = edge_index, edge_attr = edge_attr)
        out = self.conv(x = x2, edge_index = edge_index, edge_attr = edge_attr)
        return out
        

def sr_loss(data, out, K):
    power = out[:,2]
    power = torch.reshape(power, (-1, K, 1)) 
    
    abs_H_2 = data.y#torch.pow(data.y, 2)
    abs_H_2 = abs_H_2.permute(0,2,1)
    rx_power = torch.mul(abs_H_2, power)
    mask = torch.eye(K)
    mask = mask.to(device)
    valid_rx_power = torch.sum(torch.mul(rx_power, mask), 1)
    interference = torch.sum(torch.mul(rx_power, 1 - mask), 1) + var
    rate = torch.log2(1 + torch.div(valid_rx_power, interference))
    #w_rate = torch.mul(data.pos,rate)
    sum_rate = torch.mean(torch.sum(rate, 1))
    loss = torch.neg(sum_rate)
    return loss

def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = sr_loss(data,out,train_K)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / train_layouts

def test():
    model.eval()

    total_loss = 0
    for data in test_loader:
        data = data.to(device)
        with torch.no_grad():
            start = time.time()
            out = model(data)
            end = time.time()
            print('dnn time',end-start)
            loss = sr_loss(data,out,test_K)
            total_loss += loss.item() * data.num_graphs
    power = out[:,2]
    power = torch.reshape(power, (-1, test_K)) 
    Y = power.cpu().numpy()
    rates = helper_functions.compute_rates(test_config, 
            Y, directLink_channel_losses, crossLink_channel_losses)
    sr = np.mean(np.sum(rates,axis=1))
    print('actual_rates:',sr)
    
    return total_loss / test_layouts


In [2]:
train_K = 50
test_K = 50
train_layouts = 20000
test_layouts = 500

train_config = init_parameters()
var = train_config.output_noise_power / train_config.tx_power
layouts, train_dists = wg.generate_layouts(train_config, train_layouts)
train_path_losses = wg.compute_path_losses(train_config, train_dists)
train_channel_losses = helper_functions.add_shadowing(train_path_losses)
train_channel_losses = helper_functions.add_fast_fading(train_channel_losses)
train_losses = proc_train_losses(train_path_losses, train_channel_losses)

test_config = init_parameters()
layouts, test_dists = wg.generate_layouts(test_config, test_layouts)
test_path_losses = wg.compute_path_losses(test_config, test_dists)
test_channel_losses = helper_functions.add_shadowing(test_path_losses)
test_channel_losses = helper_functions.add_fast_fading(test_channel_losses)

<<<<<<<<<<<<<20000 layouts: 50_links_1000X1000_2_65_length>>>>>>>>>>>>


<<<<<<<<<<<<<500 layouts: 50_links_1000X1000_2_65_length>>>>>>>>>>>>


In [46]:
import torch

def batch_WMMSE_GPU(p_int, alpha, H, Pmax, var_noise):
    # Auto-convert inputs and move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Convert inputs to tensors and remove singleton dimensions
    p_int = torch.as_tensor(p_int, dtype=torch.float32).to(device)
    alpha = torch.as_tensor(alpha, dtype=torch.float32).to(device)
    H = torch.as_tensor(H, dtype=torch.float32).to(device)
    Pmax = torch.as_tensor(Pmax, dtype=torch.float32).to(device)
    var_noise = torch.as_tensor(var_noise, dtype=torch.float32).to(device)

    N, K, _ = p_int.shape
    b = torch.sqrt(p_int)
    mask = torch.eye(K, device=device)
    
    # Initialize tensors with proper dimensions
    f = torch.zeros((N, K, 1), device=device)
    w = torch.zeros((N, K, 1), device=device)
    
    # Initial calculations
    rx_power = H * b  # (N,K,K)
    valid_rx_power = (rx_power * mask).sum(dim=1)  # (N,K)
    interference = rx_power.square().sum(dim=2) + var_noise
    f = (valid_rx_power / interference)
    w = 1 / (1 - f * valid_rx_power)
    
    for _ in range(100):
        fp = f.unsqueeze(1) 
        H_T = H.permute(0, 2, 1) 
        rx_power = H_T * fp  
        
        valid_rx_power = (rx_power * mask).sum(1).squeeze(-1)
        bup = alpha * w * valid_rx_power
        
        wp = w.unsqueeze(1)  
        rx_power_sq = rx_power.square()
        bdown = (alpha.unsqueeze(1) * rx_power_sq * wp).sum(2)
        
        b = torch.clamp(bup / (bdown + 1e-10), 0, torch.sqrt(Pmax))
        

        rx_power = H * b.unsqueeze(1)
        valid_rx_power = (rx_power * mask).sum(1)
        interference = rx_power.square().sum(2) + var_noise
        f = (valid_rx_power / interference)
        w = 1 / (1 - f * valid_rx_power)

    return b.square().cpu().detach().numpy()

In [47]:
import time
norm_train_dists, norm_test_dists = normalize_data(1/train_dists,1/test_dists)
norm_train_losses, norm_test_losses = normalize_data(np.sqrt(train_channel_losses),np.sqrt(test_channel_losses) )
train_data_list = proc_data(train_channel_losses, train_dists, norm_train_dists, norm_train_losses, train_K)
test_data_list = proc_data(test_channel_losses, test_dists, norm_test_dists, norm_test_losses, test_K)


directLink_channel_losses = helper_functions.get_directLink_channel_losses(test_channel_losses)
crossLink_channel_losses = helper_functions.get_crossLink_channel_losses(test_channel_losses)

Pini = np.random.rand(test_layouts,test_K,1 )
start_time = time.time()
Y3 = batch_WMMSE_GPU(Pini,np.ones([test_layouts, test_K]),np.sqrt(test_channel_losses),1,var)
end_time = time.time()
time2 = end_time - start_time

Y1 = FP_optimize(test_config,test_channel_losses,np.ones([test_layouts, test_K]))
start_time = time.time()
Y2 = wf.batch_WMMSE2(Pini,np.ones([test_layouts, test_K]),np.sqrt(test_channel_losses),1,var)
end_time = time.time()
time1 = end_time - start_time




rates1 = helper_functions.compute_rates(test_config, 
            Y1, directLink_channel_losses, crossLink_channel_losses)
rates2 = helper_functions.compute_rates(test_config, 
            Y2, directLink_channel_losses, crossLink_channel_losses)
rates3 = helper_functions.compute_rates(test_config, 
            Y3, directLink_channel_losses, crossLink_channel_losses)


sr1 = np.mean(np.sum(rates1,axis=1))
sr2 = np.mean(np.sum(rates2,axis=1))
sr3 = np.mean(np.sum(rates3,axis=1))
print('FPlinQ fade:',sr1)
print('WMMSE fade:',sr2, 'time', time1)
print('WMMSE_GPU fade:',sr3, 'time', time2)

bl_Y = simple_greedy(test_channel_losses,np.ones([test_layouts, test_K]),Y1)

rates_bl = helper_functions.compute_rates(test_config, 
            bl_Y, directLink_channel_losses, crossLink_channel_losses)
sr_bl = np.mean(np.sum(rates_bl,axis=1))
print('baseline:',sr_bl)

FPlinQ fade: 199.6456248420678
WMMSE fade: 200.16008775884217 time 0.9996109008789062
WMMSE_GPU fade: 200.1567671220542 time 0.028956890106201172
baseline: 159.62223337615055


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IGCNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)
train_loader = DataLoader(train_data_list, batch_size=100, shuffle=True,num_workers=1)
test_loader = DataLoader(test_data_list, batch_size=test_layouts, shuffle=False, num_workers=1)
for epoch in range(1, 500):
    loss1 = train()
    if(epoch % 50 == 0):
        loss2 = test()
        print('Epoch {:03d}, Train Loss: {:.4f}, Val Loss: {:.4f}'.format(
            epoch, loss1, loss2))
    #scheduler.step()

dnn time 0.003275632858276367
actual_rates: 190.94908855569543
Epoch 050, Train Loss: -191.6898, Val Loss: -190.9491
dnn time 0.0033636093139648438
actual_rates: 191.96558819182297
Epoch 100, Train Loss: -192.4063, Val Loss: -191.9656
dnn time 0.003934621810913086
actual_rates: 192.0361204736302
Epoch 150, Train Loss: -192.6345, Val Loss: -192.0361
dnn time 0.003245830535888672
actual_rates: 192.29132791042463
Epoch 200, Train Loss: -192.8351, Val Loss: -192.2913
dnn time 0.003136873245239258
actual_rates: 192.25162173938332
Epoch 250, Train Loss: -192.9529, Val Loss: -192.2516
dnn time 0.003259420394897461
actual_rates: 192.17358953595198
Epoch 300, Train Loss: -192.7643, Val Loss: -192.1736
